In [ ]:
!pip install pyspark psycopg2-binary


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, avg, desc, col, explode, to_date
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import split, explode, to_date
from pyspark.sql.functions import length
import time


# Créer une SparkSession pour le traitement par lots des données historiques
spark = SparkSession.builder \
    .appName("BatchProcessingMastodon") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.2.25") \
    .getOrCreate()

# Paramètres de connexion à PostgreSQL
db_host = "some-postgres"
db_port = "5432"
db_name = "mastodon_data"
db_user = "postgres"
db_password = "mysecretpassword"
db_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}"
db_properties = {
    "user": db_user,
    "password": db_password,
    "driver": "org.postgresql.Driver"
}

# Charge les données historiques depuis PostgreSQL
print("Chargement des données historiques...")
filtered_toots_df = spark.read.jdbc(url=db_url, table="filtered_toots", properties=db_properties)

# Vérifie le schéma et afficher quelques lignes
filtered_toots_df.printSchema()
filtered_toots_df.show(5)

# Filtre les toots basés sur l'activité des utilisateurs 
X = 2  

# Calcule le nombre de toots par utilisateur
user_toot_counts = filtered_toots_df.groupBy("user_id").agg(count("*").alias("toot_count"))

# Filtre les utilisateurs avec plus de X toots
active_users = user_toot_counts.filter(col("toot_count") > X)

print(f"Utilisateurs avec plus de {X} toots :")
active_users.show()

# Join avec les toots originaux pour obtenir les toots de ces utilisateurs
active_users_toots = filtered_toots_df.join(active_users, on="user_id", how="inner")


# Extrai la date à partir du timestamp
toots_with_date = active_users_toots.withColumn("date", to_date(col("timestamp")))

toots_with_hashtags = toots_with_date.withColumn("hashtag", explode(split(col("hashtags"), ",")))

# Calcule le nombre total de toots par date et hashtag
toots_by_date_hashtag = toots_with_hashtags.groupBy("date", "hashtag").agg(count("*").alias("total_toots"))

print("Nombre total de toots par date et hashtag :")
toots_by_date_hashtag.orderBy(desc("total_toots")).show()

# Identifie le hashtag le plus fréquent
most_frequent_hashtag = toots_with_hashtags.groupBy("hashtag").agg(count("*").alias("hashtag_count")) \
    .orderBy(desc("hashtag_count")).limit(1)

print("Hashtag le plus fréquent :")
most_frequent_hashtag.show()


# Compte le nombre total de toots par jour
total_toots_per_day = filtered_toots_df.withColumn("date", to_date(col("timestamp"))) \
    .groupBy("date").agg(count("*").alias("total_toots")) \
    .orderBy("date")

print("Nombre total de toots par jour :")
total_toots_per_day.show()

filtered_toots_df = filtered_toots_df.withColumn("text_length", length(col("text")))

average_toot_length = filtered_toots_df.agg(avg("text_length").alias("average_toot_length"))

print("Longueur moyenne des toots :")
average_toot_length.show()



# Mesure les performances avant optimisation

start_time = time.time()
total_toots = filtered_toots_df.count()
end_time = time.time()
print(f"Nombre total de toots : {total_toots}")
print(f"Temps sans optimisation : {end_time - start_time} secondes")


repartitioned_df = filtered_toots_df.repartition(8)  

start_time = time.time()
total_toots = repartitioned_df.count()
end_time = time.time()
print(f"Nombre total de toots (après repartition): {total_toots}")
print(f"Temps avec repartition : {end_time - start_time} secondes")


filtered_toots_df.cache()

start_time = time.time()
total_toots = filtered_toots_df.count()
end_time = time.time()
print(f"Nombre total de toots (après cache): {total_toots}")
print(f"Temps avec cache : {end_time - start_time} secondes")


print("Analyse et optimisation terminées.")

# Arrête la SparkSession
spark.stop()


Chargement des données historiques...
root
 |-- toot_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- reblogs_count: integer (nullable = true)
 |-- favourites_count: integer (nullable = true)
 |-- replies_count: integer (nullable = true)

+------------------+-------------------+--------------------+------------------+--------+--------+-------------+----------------+-------------+
|           toot_id|          timestamp|                text|           user_id|language|hashtags|reblogs_count|favourites_count|replies_count|
+------------------+-------------------+--------------------+------------------+--------+--------+-------------+----------------+-------------+
|113282845685250307|2024-10-10 11:17:26|a novel llmbased ...|109730266331128291|      en|        |            0|               0|            0|
|